In [26]:
using SQLite
import SQLite.Stmt
import SQLite.tables
import SQLite.columns
import SQLite.DBInterface.execute
using Distributions

In [2]:
db1 = SQLite.DB("output/before/sweep_db_gathered.sqlite")
db2 = SQLite.DB("output/after/sweep_db_gathered.sqlite")

SQLite.DB("output/after/sweep_db_gathered.sqlite")

In [34]:
columns(db1, "param_combos")[:name][2:end]

9-element Vector{String}:
 "biting_rate"
 "immigration_rate_fraction"
 "immunity_loss_rate"
 "n_genes_initial"
 "switching_rate_A"
 "switching_rate_BC"
 "ectopic_recombination_rate_A"
 "ectopic_recombination_rate_BC"
 "functionality_BC"

In [3]:
import StatsAPI.pvalue
import StatsBase.mean
import StatsBase.mad
import HypothesisTests.ApproximateTwoSampleKSTest

# Functions to compare using K-S tests

"""
    Do K-S tests using SQLite queries that take one parameter, run_id.

    `db1` and `db2` are assumed to have identical parameter combinations labeled the same way,
    via the `combo_id` column in the `runs` table.

    The query should return a single value for the provided run_id.

    If values from db2 need to be extracted with a different query, provide a value for `q2`.
"""
function compare(db1, db2, q1; q2 = q1)
    combo_ids = get_combo_ids(db1)
    @assert all(combo_ids .== get_combo_ids(db2))
    [(combo_id, compare(db1, db2, combo_id, q1; q2 = q2)) for combo_id in combo_ids]
end

function get_combo_ids(db)
    [combo_id for (combo_id,) in execute(db, "SELECT DISTINCT combo_id FROM runs ORDER BY combo_id")]
end

function get_run_ids(db, combo_id)
    [run_id for (run_id,) in execute(db, "SELECT run_id FROM runs WHERE combo_id = ?", [combo_id])]
end

function compare(db1, db2, combo_id, q1; q2 = q1)
    v1 = query_values_for_combo_id(db1, combo_id, q1)
    mean1 = mean(v1)
    mad1 = mad(v1; center = mean1)

    v2 = query_values_for_combo_id(db2, combo_id, q2)
    mean2 = mean(v2)
    mad2 = mad(v2; center = mean2)

    (ApproximateTwoSampleKSTest(v1, v2), (mean1, mad1), (mean2, mad2))
end

function query_values_for_combo_id(db, combo_id, q)
    stmt = Stmt(db, q)
    run_ids = get_run_ids(db, combo_id)

    [query_value_for_run_id(stmt, run_id) for run_id in run_ids]
end

function query_value_for_run_id(stmt, run_id)
    for (value,) in execute(stmt, (run_id,))
        return value
    end
end

query_value_for_run_id (generic function with 1 method)

In [5]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT value FROM run_meta WHERE key = \"elapsed_time\" AND run_id = ?"
)
    println("Testing elapsed time for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean2 - mean1) / mean1)")
end

Testing elapsed time for parameter combination 1...
    (mean1 276.54106666666667, mad1 7.190719599900106)
    (mean2 445.5519999999999, mad2 10.197338058881677)
    p-value: 6.118046410036549e-7
    DIFFERENT distributions w/ p < 0.01
    fractional difference: 0.61116034363552
Testing elapsed time for parameter combination 2...
    (mean1 1975.3434666666665, mad1 10.734731943015476)
    (mean2 2451.4577999999997, mad2 19.502446102666983)
    p-value: 6.118046410036549e-7
    DIFFERENT distributions w/ p < 0.01
    fractional difference: 0.24102863191521928
Testing elapsed time for parameter combination 3...
    (mean1 273.6126666666667, mad1 7.22422641003835)
    (mean2 440.05973333333327, mad2 7.530532028381662)
    p-value: 6.118046410036549e-7
    DIFFERENT distributions w/ p < 0.01
    fractional difference: 0.6083309983212273
Testing elapsed time for parameter combination 4...
    (mean1 489.5032, mad1 4.203473809907062)
    (mean2 694.9358666666667, mad2 14.282104851160158)
   

## Mean # active infected hosts from 90 to 100 years

In [7]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT AVG(n_infected_active) FROM summary WHERE run_id = ? AND (time >= 90 * 360) AND (time <= 100 * 360)"
)
    println("Testing mean infected for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean1 - mean2) / mean1)")
end

Testing mean infected for parameter combination 1...
    (mean1 566.8132231404959, mad1 40.515475171162066)
    (mean2 542.7283746556473, mad2 32.09200736001613)
    p-value: 0.07626242398653403
    undetectable difference between distributions w/ p < 0.01
    fractional difference: 0.042491684211959056
Testing mean infected for parameter combination 2...
    (mean1 829.3856749311293, mad1 13.192300732157038)
    (mean2 809.2567493112948, mad2 12.72995756098681)
    p-value: 0.0025452675974334303
    DIFFERENT distributions w/ p < 0.01
    fractional difference: 0.024269680835162677
Testing mean infected for parameter combination 3...
    (mean1 512.9944903581268, mad1 44.08188910284007)
    (mean2 561.7123966942148, mad2 35.86672077693866)
    p-value: 0.02805685626145722
    undetectable difference between distributions w/ p < 0.01
    fractional difference: -0.0949676989748517
Testing mean infected for parameter combination 4...
    (mean1 939.1267217630851, mad1 10.002459595372125)

## Mean # circulating genes from 90 to 100 years

In [11]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT AVG(n_circulating_genes_blood) FROM gene_strain_counts WHERE run_id = ? AND (time >= 90 * 360) AND (time <= 100 * 360)"
)
    println("Testing mean infected for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean1 - mean2) / mean1)")
end

Testing mean infected for parameter combination 1...
    (mean1 1130.4727272727273, mad1 191.4982938229782)
    (mean2 1059.7454545454543, mad2 101.19434051400086)
    p-value: 0.18130044993812283
    undetectable difference between distributions w/ p < 0.01
    fractional difference: 0.06256433350488956
Testing mean infected for parameter combination 2...
    (mean1 2587.8909090909087, mad1 171.3349036513025)
    (mean2 2518.5939393939398, mad2 114.90616466818028)
    p-value: 0.18130044993812283
    undetectable difference between distributions w/ p < 0.01
    fractional difference: 0.02677739214336205
Testing mean infected for parameter combination 3...
    (mean1 975.3151515151515, mad1 123.53221393948517)
    (mean2 1128.8909090909094, mad2 104.15954495101207)
    p-value: 0.0025452675974334303
    DIFFERENT distributions w/ p < 0.01
    fractional difference: -0.15746270047909963
Testing mean infected for parameter combination 4...
    (mean1 9083.97575757576, mad1 38.04447147365